# Variation of Earnings Beat or Earnings Miss Predictor

Nodes: A stock/ticker in a specific time (MM/YYYY), features: price, EPS, etc.

Edges: 2 kinds -- temporal and in the same plane (assume time is the same e.g. 04/2021). In the same plane, nodes are linked (edges formed) if owned by the same investor. Theoretically, edge is given a larger weight if multiple investors buy/own stock in the 2 connecting stock nodes and/or the purchase amount is large.


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
file_path = './data/politician-insider-trading-activity-apr-2021.csv'
df = pd.read_csv(file_path)
df.head()

,Transaction Date,First Name,Last Name,Title,Symbol,Name,Transaction Total,Transaction,Note Text
0,2021-04-30,Diana,Harshbarger,US Congressman,AEE,Ameren Corp,24002.0,Sell,"By Director And Spouse. Traded $3,003 To $45,000"
1,2021-04-30,Kathy,Manning,US Congressman,DIS,Walt Disney Co,8000.0,Sell,"Traded $1,001 to $15,000"
2,2021-04-30,Kathy,Manning,US Congressman,V,Visa Inc,8000.0,Sell,"Traded $1,001 to $15,000"
3,2021-04-30,Kathy,Manning,US Congressman,VTRS,Viatris Inc,8000.0,Sell,"Traded $1,001 to $15,000"
4,2021-04-30,Kathy,Manning,US Congressman,TJX,TJX Companies,8000.0,Sell,"Traded $1,001 to $15,000"


In [6]:
MONTHS = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
YEARS = ['2020', '2021', '2022', '2023']

In [7]:
# Company lists
dow_list = ['axp', 'amgn', 'aapl', 'ba', 'cat', 'cvx', 'csco', 'ko', 'dow', 'gs', 'hd', 'hon', 'ibm', 'intc', 'jnj', 'jpm', 'mcd', 'mmm', 'mrk', 'msft', 'nke', 'pg', 'crm', 'trv', 'utx', 'vz', 'v', 'wba', 'wmt', 'dis']
# Create a list of the ticker symbols for the S&P 500 companies
sp500_list = ['a', 'aapl', 'abbv', 'abt', 'acn', 'adbe', 'aig', 'all', 'amgn', 'amzn', 'axp', 'ba', 'bac', 'biib', 'bk', 'bkng', 'blk', 'bmy', 'brk.b', 'bsx', 'cat', 'chtr', 'cl', 'cmcsa', 'cof', 'cop', 'cost', 'crm', 'csco', 'cvx', 'dhr', 'dis', 'dow', 'duke', 'emr', 'exc', 'f', 'fb', 'fdx', 'gd', 'ge', 'gild', 'gm', 'goog', 'googl', 'gs', 'hd', 'hon', 'ibm', 'intc', 'jnj', 'jpm', 'kmi', 'ko', 'lly', 'lmt', 'low', 'ma', 'mcd', 'mdlz', 'met', 'microsoft', 'mmm', 'mo', 'mrk', 'ms', 'msft', 'nflx', 'nke', 'nvda', 'orcl', 'pep', 'pfe', 'pg', 'pm', 'pypl', 'qcom', 'crm', 'sbux', 'slb', 'so', 'spg', 't', 'tgt', 'tmo', 'tpr', 'trv', 'tsla', 'txn', 'unh', 'unp', 'ups', 'usb', 'v', 'vz', 'wba', 'wfc', 'wmt', 'xom']
# Create a list of the ticker symbols for the NASDAQ 100 companies
nasdaq100_list = ['aapl', 'adbe', 'adi', 'adp', 'adsk', 'alxn', 'amat', 'amd', 'amgn', 'amzn', 'asml', 'atvi', 'avgo', 'bidu', 'biib', 'bkng', 'bmrn', 'bngo', 'cdns', 'celg', 'cern', 'chkp', 'chtr', 'cmcsa', 'cost', 'csco', 'csct', 'ctas', 'ctrp', 'ctsh', 'ctxs', 'dltr', 'ea', 'ebay', 'esrx', 'fast', 'fb', 'fisv', 'fox', 'foxa', 'gild', 'goog', 'googl', 'has', 'hsic', 'idxx', 'ilmn', 'incy', 'intc', 'intu', 'isrg', 'jd', 'khc', 'klac', 'lbtya', 'lbtym', 'liberty', 'lrcx', 'mar', 'mat', 'mchp', 'mdlz', 'meli', 'mnst', 'msft', 'mu', 'mxim', 'myl', 'nclh', 'nflx', 'ntap', 'ntes', 'nvda', 'nxpi', 'orly', 'payx', 'pcar', 'pypl', 'qcom', 'regn', 'rost', 'sbux', 'sirius', 'swks', 'symc', 'tmus', 'trip', 'tsco', 'tsem', 'txn', 'ulta', 'vrtx', 'vrsn', 'vrtx', 'wdc', 'wday', 'wltw', 'wynn', 'xlnx', 'xray', 'zm']


In [11]:
import torch
import torch.nn as nn
import torch_geometric.nn as gnn
import torch_geometric.data as gdata
import numpy as np
import h5py
import os
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import datetime as dt
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import time
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix, classification_report
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'sentence_transformers'